In [1]:


import torch
import os

import rlcard
from rlcard.agents import RandomAgent,DQN_agent,DQN_conf
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

# Make environment
env = rlcard.make('blackjack', config={'seed': 0})
eval_env = rlcard.make('blackjack', config={'seed': 0})

# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 100
evaluate_num = 1000
episode_num = 10000

# The intial memory size
memory_init_size = 1000

# Train the agent every X steps
train_every = 1

# The paths for saving the logs and learning curves
log_dir = './experiments/blackjack_results_dqn/'

# Set a global seed
set_global_seed(0)

params = {
    "scope":"DQN-Agent",
    "num_actions":env.action_num,
    "replay_memory_size":memory_init_size,
    "num_states":env.state_shape,
    "discount_factor" :0.99,
    "epsilon_start" : 1.0,
    "epsilon_end" : 0.1,
    "epsilon_decay_steps":20000,
    "batch_size":32,
    "train_every":1,
    "mlp_layers":[128,128],
    "lr":0.1,
}

agent_conf = DQN_conf(**params)
agent = DQN_agent(agent_conf)

random_agent = RandomAgent(action_num=eval_env.action_num)
env.set_agents([agent, random_agent])
eval_env.set_agents([agent, random_agent])

logger = Logger(log_dir)

for episode in range(episode_num):

    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.feed(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

# Close files in the logger
logger.close_files()

# Plot the learning curve
logger.plot('DQN')

# Save model
save_dir = 'models/leduc_holdem_dqn_pytorch'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
state_dict = agent.get_state_dict()
print(state_dict. keys())
torch.save(state_dict, os.path.join(save_dir, 'model.pth'))





----------------------------------------
  timestep     |  1
  reward       |  -1.0
----------------------------------------


AttributeError: 'DQN_agent' object has no attribute 'scope'

In [1]:
''' An example of learning a Deep-Q Agent on Leduc Holdem
'''
import torch
import os

import rlcard
from rlcard.agents import DQNAgentPytorch as DQNAgent
from rlcard.agents import RandomAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

# Make environment
env = rlcard.make('blackjack', config={'seed': 0})
eval_env = rlcard.make('blackjack', config={'seed': 0})

# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 100
evaluate_num = 1000
episode_num = 100000

# The intial memory size
memory_init_size = 1000

# Train the agent every X steps
train_every = 1

# The paths for saving the logs and learning curves
log_dir = './experiments/limit_holdem_dqn_result/'

# Set a global seed
set_global_seed(0)

agent = DQNAgent(scope='dqn',
                 action_num=env.action_num,
                 replay_memory_init_size=memory_init_size,
                 train_every=train_every,
                 state_shape=env.state_shape,
                 mlp_layers=[128, 128],
                 device=torch.device('cpu'))
random_agent = RandomAgent(action_num=eval_env.action_num)
env.set_agents([agent, random_agent])
eval_env.set_agents([agent, random_agent])

# Init a Logger to plot the learning curve
logger = Logger(log_dir)

for episode in range(episode_num):

    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.feed(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

# Close files in the logger
logger.close_files()

# Plot the learning curve
logger.plot('DQN')

# Save model
save_dir = 'models/leduc_holdem_dqn_pytorch'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
state_dict = agent.get_state_dict()
print(state_dict. keys())
torch.save(state_dict, os.path.join(save_dir, 'model.pth'))





----------------------------------------
  timestep     |  1
  reward       |  -0.956
----------------------------------------

----------------------------------------
  timestep     |  133
  reward       |  -0.956
----------------------------------------

----------------------------------------
  timestep     |  263
  reward       |  -0.952
----------------------------------------

----------------------------------------
  timestep     |  396
  reward       |  -0.932
----------------------------------------

----------------------------------------
  timestep     |  539
  reward       |  -0.942
----------------------------------------

----------------------------------------
  timestep     |  677
  reward       |  -0.952
----------------------------------------

----------------------------------------
  timestep     |  824
  reward       |  -0.942
----------------------------------------

----------------------------------------
  timestep     |  968
  reward       |  -0.93
---

KeyboardInterrupt: 

In [68]:

import rlcard
from rlcard.agents import RandomAgent as RandomAgent
from rlcard.agents import BlackjackHumanAgent as HumanAgent
from rlcard.utils.utils import print_card

# Make environment and enable human mode
# Set 'record_action' to True because we need it to print results
player_num = 2
env = rlcard.make('blackjack', config={'record_action': True, 'game_player_num': player_num})
human_agent = HumanAgent(env.action_num)
random_agent = RandomAgent(env.action_num)
env.set_agents([human_agent, random_agent])

print(">> Blackjack human agent")

while (True):
    print(">> Start a new game")

    trajectories, payoffs = env.run(is_training=False)
    # If the human does not take the final action, we need to
    # print other players action

    if len(trajectories[0]) != 0:
        final_state = []
        action_record = []
        state = []
        _action_list = []

        for i in range(player_num):
            final_state.append(trajectories[i][-1][-2])
            state.append(final_state[i]['raw_obs'])

        action_record.append(final_state[i]['action_record'])
        for i in range(1, len(action_record) + 1):
            _action_list.insert(0, action_record[-i])

        for pair in _action_list[0]:
            print('>> Player', pair[0], 'chooses', pair[1])

    # Let's take a look at what the agent card is
    print('===============   Dealer hand   ===============')
    print_card(state[0]['state'][1])

    for i in range(player_num):
        print('===============   Player {} Hand   ==============='.format(i))
        print_card(state[i]['state'][0])

    print('===============     Result     ===============')
    for i in range(player_num):
        if payoffs[i] == 1:
            print('Player {} win {} chip!'.format(i, payoffs[i]))
        elif payoffs[i] == 0:
            print('Player {} is tie'.format(i))
        else:
            print('Player {} lose {} chip!'.format(i, -payoffs[i]))
        print('')

    input("Press any key to continue...")


>> Blackjack human agent
>> Start a new game

=============   Dealer Hand   ===============
┌─────────┐
│A        │
│         │
│         │
│    ♥    │
│         │
│         │
│        A│
└─────────┘
===============   Player 0 Hand   ===============
┌─────────┐   ┌─────────┐
│4        │   │9        │
│         │   │         │
│         │   │         │
│    ♦    │   │    ♣    │
│         │   │         │
│         │   │         │
│        4│   │        9│
└─────────┘   └─────────┘
===============   Player 1 Hand   ===============
┌─────────┐   ┌─────────┐
│10       │   │J        │
│         │   │         │
│         │   │         │
│    ♠    │   │    ♣    │
│         │   │         │
│         │   │         │
│       01│   │        J│
└─────────┘   └─────────┘

=========== Actions You Can Choose ===========
0: hit, 1: stand

>> Player 0 chooses stand
>> Player 1 chooses stand
===============   Dealer hand   ===============
┌─────────┐   ┌─────────┐
│10       │   │A        │
│         │   

ValueError: invalid literal for int() with base 10: ''

In [ ]:
0